In [1]:
# Dependencies
from bs4 import BeautifulSoup 
import requests
import pandas as pd
from splinter import Browser
from selenium import webdriver
import time

In [2]:
#set up path for windows
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
url="https://www.espn.com/college-football/teams"
browser.visit(url)
html=browser.html
soup=BeautifulSoup(html, 'lxml')
browser.quit()

In [4]:
stat_link=soup.find_all('a', class_="AnchorLink")
team={
    "name":[],
    "url":[]
}
name_pars=""


for alink in stat_link:
    if alink.text=="Stats":
        length=0
        team["url"].append("https://www.espn.com"+alink.get("href"))
        name_pars=alink.get("href").split('/')
        length=len(name_pars)
        length=length-1
        team["name"].append(name_pars[length])
                


In [5]:
df=pd.DataFrame(team)
df=df[df.name != 'stats']
df.to_csv("Team_Stat_URL.csv")

In [6]:
import pymongo
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [7]:
db = client.ncaa_football_stats
collection = db.players



In [8]:
print(collection)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ncaa_football_stats'), 'players')


In [9]:
browser.visit("https://www.espn.com/college-football/team/stats/_/id/96/kentucky-wildcats")

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=62813): Max retries exceeded with url: /session/128eafa39055f23aa9cb9c144d0615dc/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001C9F02F09E8>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it',))

In [10]:
def get_stat_tables(url):
    print(url)
    browser.visit(player)
    browser.is_element_present_by_css("stats-cell", wait_time=10)
    html2 = browser.html
    player_stats1 = BeautifulSoup(html2, 'lxml')
    return pd.read_html(player_stats1)
    browser.quit()

In [11]:
df["stat_tables"] = df["url"].map(get_stat_tables)

https://www.espn.com/college-football/team/stats/_/id/2132/cincinnati-bearcats


NameError: name 'player' is not defined

In [ ]:

browser2 = Browser('chrome', **executable_path, headless=False)
for player in df["url"]:
    length=0
    name_pars1=str(player).split('/')
    length=len(name_pars1)
    length=length-1
    team_name=name_pars1[length]
    print(team_name)
    print(player)
    
    browser2.visit(player)
    time.sleep(10)
    html2 = browser2.html
    with open(team_name+".html", "w") as fwrite:
        fwrite.write(html2)
    player_stats1 = BeautifulSoup(html2, 'lxml')
    print(player_stats1) 
    stats_tables = pd.read_html(player_stats1)
     print(stats_tables)
    
    passing=pd.merge(stats_tables[0], stats_tables[1],  left_index=True, right_index=True)
    print (passing)
    rushing=pd.merge(stats_tables[2], stats_tables[3],  left_index=True, right_index=True)
    print (rushing)
    receiving=pd.merge(stats_tables[4], stats_tables[5],  left_index=True, right_index=True)
    print (receiving)
    team_data={ "name":team_name,
        "passing":passing.to_json(),
        "rushing":rushing.to_json(),
        "receiving":receiving.to_json()
    
        }
    
    collection.update({"name": team_data["name"]}, {"$set": team_data}, upsert=True)
    browser.quit()
    browser2.quit()
    break
   
    
    
    
   

    

In [ ]:
passing = stats_tables[0]
passing_new = passing.append(stats_tables[1],ignore_index=True)


df.to_json(r'C:\Users\Poona\Desktop\Passing_DataFrame.json')

import json
from pymongo import MongoClient


client = MongoClient('localhost', 27017)
db = client['passing_db']
collection_passing = db['passing']

with open('C:\\Users\\Poona\\Desktop\\Passing_DataFrame.json') as f:
    file_data = json.load(f)

# use collection_passing.insert(file_data) if pymongo version < 3.0
collection_passing.insert_many(file_data)  
client.close()